# Portfolio Optimization - Cleaned Version
## Essential cells only for sector rotation strategy analysis

In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Portfolio Optimization - Cleaned Version\n",
        "## Essential cells only for sector rotation strategy analysis"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1. Import Libraries and Setup"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 1.0 - Import necessary libraries\n",
        "!pip install fredapi -q\n",
        "import yfinance as yf\n",
        "from fredapi import Fred\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "from scipy.optimize import minimize\n",
        "import warnings\n",
        "warnings.filterwarnings('ignore')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2. Fetch Sector ETF Data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 2.0 - Fetch sector ETF data\n",
        "tickers = [\"XLK\", \"XLC\", \"XLV\", \"XLF\", \"XLI\", \"XLB\", \"XLU\", \"XLE\", \"XLY\", \"XLP\", \"XLRE\", \"SPY\"]\n",
        "\n",
        "ticker_to_sector = {\n",
        "    \"XLK\": \"Technology\",\n",
        "    \"XLC\": \"Communications\",\n",
        "    \"XLV\": \"Healthcare\",\n",
        "    \"XLF\": \"Financial\",\n",
        "    \"XLI\": \"Industrial\",\n",
        "    \"XLB\": \"Material\",\n",
        "    \"XLU\": \"Utilities\",\n",
        "    \"XLE\": \"Energy\",\n",
        "    \"XLY\": \"CDiscretionary\",\n",
        "    \"XLP\": \"CStaples\",\n",
        "    \"XLRE\": \"Real Estate\",\n",
        "    \"SPY\": \"S&P 500\"\n",
        "}\n",
        "\n",
        "start_date = \"2010-01-01\"\n",
        "end_date = \"2025-12-30\"\n",
        "\n",
        "sector_data = {}\n",
        "for ticker in tickers:\n",
        "    data = yf.download(ticker, start=start_date, end=end_date, interval=\"1wk\", auto_adjust=True, progress=False)\n",
        "    if not data.empty:\n",
        "        sector_data[ticker] = data[\"Close\"]\n",
        "\n",
        "if sector_data:\n",
        "    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())\n",
        "    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]\n",
        "    df_sector = df_sector.ffill().bfill()\n",
        "    df_sector.to_csv(\"Sector_ETF_Data.csv\")\n",
        "    print(\"Sector ETF data saved successfully!\")\n",
        "    print(f\"Data shape: {df_sector.shape}\")\n",
        "    display(df_sector.head())"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3. Fetch Macroeconomic Data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 3.0 - Fetch macroeconomic data from FRED\n",
        "fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')\n",
        "\n",
        "weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')\n",
        "\n",
        "series_dict = {\n",
        "    'GDP': 'GDP',\n",
        "    'Inflation (CPI)': 'CPIAUCSL',\n",
        "    'Unemployment Rate': 'UNRATE',\n",
        "    'Interest Rate (Fed Funds)': 'FEDFUNDS'\n",
        "}\n",
        "\n",
        "macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))\n",
        "macro_data.index.name = \"Date\"\n",
        "\n",
        "for label, series_id in series_dict.items():\n",
        "    data = fred.get_series(series_id)\n",
        "    if data is not None:\n",
        "        data = data.to_frame(name=label)\n",
        "        data.index = pd.to_datetime(data.index)\n",
        "        \n",
        "        if label == 'Interest Rate (Fed Funds)':\n",
        "            daily = data.resample('D').ffill()\n",
        "        else:\n",
        "            daily = data.resample('D').interpolate(method='linear')\n",
        "        \n",
        "        weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')\n",
        "        macro_data[label] = weekly_processed[label]\n",
        "\n",
        "macro_data.to_csv(\"Macroeconomic_Data.csv\", index=True)\n",
        "print(\"Macroeconomic Data saved successfully!\")\n",
        "display(macro_data.head())"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4. Calculate Returns and Relative Performance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 4.0 - Calculate weekly returns and rolling means\n",
        "df_sector_pct_change = df_sector.pct_change().dropna()\n",
        "rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()\n",
        "\n",
        "# Calculate relative performance vs S&P 500\n",
        "relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)\n",
        "sector_etfs = [col for col in rolling_mean_pct_change.columns if col != \"S&P 500\"]\n",
        "\n",
        "for sector in sector_etfs:\n",
        "    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change[\"S&P 500\"]\n",
        "\n",
        "print(\"Relative performance calculated!\")\n",
        "display(relative_performance.head())"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5. Generate Trading Signals"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 5.0 - Fixed threshold signals\n",
        "buy_threshold = 0.001\n",
        "sell_threshold = -0.001\n",
        "\n",
        "signals = pd.DataFrame(index=relative_performance.index)\n",
        "for sector in relative_performance.columns:\n",
        "    signals[sector] = 'Hold'\n",
        "    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'\n",
        "    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'\n",
        "\n",
        "# 5.1 - Dynamic threshold signals\n",
        "rolling_std_relative_performance = relative_performance.rolling(window=12).std()\n",
        "multiplier = 0.5\n",
        "\n",
        "dynamic_buy_thresholds = multiplier * rolling_std_relative_performance\n",
        "dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance\n",
        "\n",
        "signals_dynamic = pd.DataFrame(index=relative_performance.index)\n",
        "for sector in relative_performance.columns:\n",
        "    signals_dynamic[sector] = 'Hold'\n",
        "    signals_dynamic.loc[relative_performance[sector] > dynamic_buy_thresholds[sector], sector] = 'Buy'\n",
        "    signals_dynamic.loc[relative_performance[sector] < dynamic_sell_thresholds[sector], sector] = 'Sell'\n",
        "\n",
        "print(\"Trading signals generated!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6. Backtest Fixed Strategy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 6.0 - Backtest fixed threshold strategy\n",
        "portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])\n",
        "num_tradeable_sectors = len(sector_etfs)\n",
        "\n",
        "returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]\n",
        "signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]\n",
        "\n",
        "for i in range(len(signals_aligned)):\n",
        "    current_date = signals_aligned.index[i]\n",
        "    current_signals = signals_aligned.loc[current_date]\n",
        "    current_week_returns = returns_aligned.loc[current_date]\n",
        "    \n",
        "    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}\n",
        "    \n",
        "    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']\n",
        "    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']\n",
        "    active_sectors_this_period = buy_sectors + hold_sectors\n",
        "    \n",
        "    if len(active_sectors_this_period) > 0:\n",
        "        weight_per_active_sector = 1.0 / len(active_sectors_this_period)\n",
        "        for sector in active_sectors_this_period:\n",
        "            weights_for_this_period[sector] = weight_per_active_sector\n",
        "    \n",
        "    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)\n",
        "    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return\n",
        "\n",
        "portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1\n",
        "print(\"Fixed strategy backtest complete!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7. Backtest Dynamic Strategy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 7.0 - Backtest dynamic threshold strategy (without costs)\n",
        "portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])\n",
        "\n",
        "returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]\n",
        "signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]\n",
        "\n",
        "valid_indices = signals_aligned_dynamic.dropna(how='all').index\n",
        "signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]\n",
        "returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]\n",
        "portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]\n",
        "\n",
        "for i in range(len(signals_aligned_dynamic)):\n",
        "    current_date = signals_aligned_dynamic.index[i]\n",
        "    current_signals = signals_aligned_dynamic.loc[current_date]\n",
        "    current_week_returns = returns_aligned_dynamic.loc[current_date]\n",
        "    \n",
        "    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}\n",
        "    \n",
        "    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']\n",
        "    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']\n",
        "    active_sectors_this_period = buy_sectors + hold_sectors\n",
        "    \n",
        "    if len(active_sectors_this_period) > 0:\n",
        "        weight_per_active_sector = 1.0 / len(active_sectors_this_period)\n",
        "        for sector in active_sectors_this_period:\n",
        "            weights_for_this_period_dynamic[sector] = weight_per_active_sector\n",
        "    \n",
        "    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)\n",
        "    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return\n",
        "\n",
        "portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1\n",
        "\n",
        "# 7.1 - Backtest dynamic strategy with transaction costs\n",
        "portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])\n",
        "transaction_cost_rate = 0.001\n",
        "previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}\n",
        "\n",
        "for i in range(len(signals_aligned_dynamic)):\n",
        "    current_date = signals_aligned_dynamic.index[i]\n",
        "    current_signals = signals_aligned_dynamic.loc[current_date]\n",
        "    current_week_returns = returns_aligned_dynamic.loc[current_date]\n",
        "    \n",
        "    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}\n",
        "    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']\n",
        "    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']\n",
        "    active_sectors_this_period = buy_sectors + hold_sectors\n",
        "    \n",
        "    if len(active_sectors_this_period) > 0:\n",
        "        weight_per_active_sector = 1.0 / len(active_sectors_this_period)\n",
        "        for sector in active_sectors_this_period:\n",
        "            weights_for_this_period_dynamic[sector] = weight_per_active_sector\n",
        "    \n",
        "    transaction_costs = sum(abs(weights_for_this_period_dynamic[s] - previous_period_weights[s]) * transaction_cost_rate for s in sector_etfs)\n",
        "    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)\n",
        "    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs\n",
        "    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs\n",
        "    previous_period_weights = weights_for_this_period_dynamic\n",
        "\n",
        "portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1\n",
        "print(\"Dynamic strategy backtests complete!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8. Backtest Monthly Strategy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 8.0 - Prepare monthly data and backtest\n",
        "df_sector_monthly = df_sector.resample('ME').last()\n",
        "df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()\n",
        "rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()\n",
        "\n",
        "relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)\n",
        "sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != \"S&P 500\"]\n",
        "\n",
        "for sector in sector_etfs_monthly:\n",
        "    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change[\"S&P 500\"]\n",
        "\n",
        "signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)\n",
        "for sector in relative_performance_monthly.columns:\n",
        "    signals_monthly[sector] = 'Hold'\n",
        "    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold, sector] = 'Buy'\n",
        "    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold, sector] = 'Sell'\n",
        "\n",
        "portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])\n",
        "returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]\n",
        "signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]\n",
        "\n",
        "for i in range(len(signals_aligned_monthly)):\n",
        "    current_date = signals_aligned_monthly.index[i]\n",
        "    current_signals = signals_aligned_monthly.loc[current_date]\n",
        "    current_month_returns = returns_aligned_monthly.loc[current_date]\n",
        "    \n",
        "    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}\n",
        "    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']\n",
        "    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']\n",
        "    active_sectors_this_period = buy_sectors + hold_sectors\n",
        "    \n",
        "    if len(active_sectors_this_period) > 0:\n",
        "        weight_per_active_sector = 1.0 / len(active_sectors_this_period)\n",
        "        for sector in active_sectors_this_period:\n",
        "            weights_for_this_period_monthly[sector] = weight_per_active_sector\n",
        "    \n",
        "    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)\n",
        "    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return\n",
        "\n",
        "portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1\n",
        "print(\"Monthly strategy backtest complete!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 9. Calculate Performance Metrics"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 9.0 - Calculate all performance metrics\n",
        "rf = 0.01\n",
        "\n",
        "# Fixed Strategy\n",
        "num_weeks = len(portfolio_returns)\n",
        "num_years = num_weeks / 52.0\n",
        "portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]\n",
        "portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1\n",
        "portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)\n",
        "portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility\n",
        "\n",
        "# S&P 500\n",
        "s_p_500_returns_aligned = df_sector_pct_change[\"S&P 500\"].align(portfolio_returns, join='inner', axis=0)[0]\n",
        "s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1\n",
        "s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]\n",
        "s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1\n",
        "s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)\n",
        "s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility\n",
        "\n",
        "# Dynamic Strategy\n",
        "num_weeks_dynamic = len(portfolio_returns_dynamic)\n",
        "num_years_dynamic = num_weeks_dynamic / 52.0\n",
        "portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]\n",
        "portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1\n",
        "portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)\n",
        "portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic\n",
        "\n",
        "# Dynamic Strategy with Costs\n",
        "num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)\n",
        "num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0\n",
        "portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]\n",
        "portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1\n",
        "portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)\n",
        "portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs\n",
        "\n",
        "# Monthly Strategy\n",
        "num_months_monthly = len(portfolio_returns_monthly)\n",
        "num_years_monthly = num_months_monthly / 12.0\n",
        "portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]\n",
        "portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1\n",
        "portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)\n",
        "portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly\n",
        "\n",
        "# Print Summary\n",
        "print(\"\\n--- Strategy Performance Comparison ---\")\n",
        "print(f\"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       \")\n",
        "print(f\"------------------------------------------------------------------------------------------------------------------\")\n",
        "print(f\"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       \")\n",
        "print(f\"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       \")\n",
        "print(f\"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       \")\n",
        "print(f\"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       \")\n",
        "print(\"------------------------------------------------------------------------------------------------------------------\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 10. Visualize Results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 10.0 - Plot cumulative returns comparison\n",
        "plt.figure(figsize=(14, 7))\n",
        "plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy', color='orange', linewidth=2)\n",
        "plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy', color='blue', linewidth=2)\n",
        "plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy (no costs)', color='green', linewidth=2)\n",
        "plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy (with costs)', color='purple', linewidth=2)\n",
        "plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500', color='red', linewidth=2)\n",
        "plt.title('Cumulative Returns: All Strategies vs. S&P 500', fontsize=16, fontweight='bold')\n",
        "plt.xlabel('Date', fontsize=12)\n",
        "plt.ylabel('Cumulative Return', fontsize=12)\n",
        "plt.grid(True, linestyle='--', alpha=0.6)\n",
        "plt.legend(loc='best', fontsize=10)\n",
        "plt.tight_layout()\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 11. Calculate Trade Frequency"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# 11.0 - Calculate weekly trade frequency\n",
        "fixed_strategy_weekly_trades = pd.Series(index=signals_aligned.index, dtype=int)\n",
        "dynamic_strategy_weekly_trades = pd.Series(index=signals_aligned_dynamic.index, dtype=int)\n",
        "\n",
        "for i in range(len(signals_aligned)):\n",
        "    if i == 0:\n",
        "        fixed_strategy_weekly_trades.iloc[i] = 0\n",
        "    else:\n",
        "        num_changes = (signals_aligned.iloc[i] != signals_aligned.iloc[i-1]).sum()\n",
        "        fixed_strategy_weekly_trades.iloc[i] = num_changes\n",
        "\n",
        "for i in range(len(signals_aligned_dynamic)):\n",
        "    if i == 0:\n",
        "        dynamic_strategy_weekly_trades.iloc[i] = 0\n",
        "    else:\n",
        "        num_changes = (signals_aligned_dynamic.iloc[i] != signals_aligned_dynamic.iloc[i-1]).sum()\n",
        "        dynamic_strategy_weekly_trades.iloc[i] = num_changes\n",
        "\n",
        "average_fixed_weekly_trades = fixed_strategy_weekly_trades.mean()\n",
        "average_dynamic_weekly_trades = dynamic_strategy_weekly_trades.mean()\n",
        "\n",
        "print(f\"\\nAverage Weekly Trades (Fixed-Threshold Strategy): {average_fixed_weekly_trades:.2f}\")\n",
        "print(f\"Average Weekly Trades (Dynamic-Threshold Strategy): {average_dynamic_weekly_trades:.2f}\")"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.8.0"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 4
}

## 1. Import Libraries and Setup

In [ ]:
# 1.0 - Import necessary libraries
!pip install fredapi -q
import yfinance as yf
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

## 2. Fetch Sector ETF Data

In [ ]:
# 2.0 - Fetch sector ETF data
tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "XLRE", "SPY"]

ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "XLRE": "Real Estate",
    "SPY": "S&P 500"
}

start_date = "2010-01-01"
end_date = "2025-12-30"

sector_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

if sector_data:
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]
    df_sector = df_sector.ffill().bfill()
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully!")
    print(f"Data shape: {df_sector.shape}")
    display(df_sector.head())

## 3. Fetch Macroeconomic Data

In [ ]:
# 3.0 - Fetch macroeconomic data from FRED
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is not None:
        data = data.to_frame(name=label)
        data.index = pd.to_datetime(data.index)
        
        if label == 'Interest Rate (Fed Funds)':
            daily = data.resample('D').ffill()
        else:
            daily = data.resample('D').interpolate(method='linear')
        
        weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
        macro_data[label] = weekly_processed[label]

macro_data.to_csv("Macroeconomic_Data.csv", index=True)
print("Macroeconomic Data saved successfully!")
display(macro_data.head())

## 4. Calculate Returns and Relative Performance

In [ ]:
# 4.0 - Calculate weekly returns and rolling means
df_sector_pct_change = df_sector.pct_change().dropna()
rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

# Calculate relative performance vs S&P 500
relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)
sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("Relative performance calculated!")
display(relative_performance.head())

## 5. Generate Trading Signals

In [ ]:
# 5.0 - Fixed threshold signals
buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)
for sector in relative_performance.columns:
    signals[sector] = 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

# 5.1 - Dynamic threshold signals
rolling_std_relative_performance = relative_performance.rolling(window=12).std()
multiplier = 0.5

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)
for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'
    signals_dynamic.loc[relative_performance[sector] > dynamic_buy_thresholds[sector], sector] = 'Buy'
    signals_dynamic.loc[relative_performance[sector] < dynamic_sell_thresholds[sector], sector] = 'Sell'

print("Trading signals generated!")

## 6. Backtest Fixed Strategy

In [ ]:
# 6.0 - Backtest fixed threshold strategy
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])
num_tradeable_sectors = len(sector_etfs)

returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date]
    current_week_returns = returns_aligned.loc[current_date]
    
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}
    
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
    
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1
print("Fixed strategy backtest complete!")

## 7. Backtest Dynamic Strategy

In [ ]:
# 7.0 - Backtest dynamic threshold strategy (without costs)
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]
    
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector
    
    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

# 7.1 - Backtest dynamic strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])
transaction_cost_rate = 0.001
previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]
    
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector
    
    transaction_costs = sum(abs(weights_for_this_period_dynamic[s] - previous_period_weights[s]) * transaction_cost_rate for s in sector_etfs)
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1
print("Dynamic strategy backtests complete!")

## 8. Backtest Monthly Strategy

In [ ]:
# 8.0 - Prepare monthly data and backtest
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold, sector] = 'Sell'

portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1
print("Monthly strategy backtest complete!")

## 9. Calculate Performance Metrics

In [ ]:
# 9.0 - Calculate all performance metrics
rf = 0.01

# Fixed Strategy
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# S&P 500
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# Dynamic Strategy
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# Dynamic Strategy with Costs
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Monthly Strategy
num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0
portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# Print Summary
print("\n--- Strategy Performance Comparison ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

## 10. Visualize Results

In [ ]:
# 10.0 - Plot cumulative returns comparison
plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy', color='orange', linewidth=2)
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy', color='blue', linewidth=2)
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy (no costs)', color='green', linewidth=2)
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy (with costs)', color='purple', linewidth=2)
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500', color='red', linewidth=2)
plt.title('Cumulative Returns: All Strategies vs. S&P 500', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc='best', fontsize=10)
plt.tight_layout()
plt.show()

## 11. Calculate Trade Frequency

In [ ]:
# 11.0 - Calculate weekly trade frequency
fixed_strategy_weekly_trades = pd.Series(index=signals_aligned.index, dtype=int)
dynamic_strategy_weekly_trades = pd.Series(index=signals_aligned_dynamic.index, dtype=int)

for i in range(len(signals_aligned)):
    if i == 0:
        fixed_strategy_weekly_trades.iloc[i] = 0
    else:
        num_changes = (signals_aligned.iloc[i] != signals_aligned.iloc[i-1]).sum()
        fixed_strategy_weekly_trades.iloc[i] = num_changes

for i in range(len(signals_aligned_dynamic)):
    if i == 0:
        dynamic_strategy_weekly_trades.iloc[i] = 0
    else:
        num_changes = (signals_aligned_dynamic.iloc[i] != signals_aligned_dynamic.iloc[i-1]).sum()
        dynamic_strategy_weekly_trades.iloc[i] = num_changes

average_fixed_weekly_trades = fixed_strategy_weekly_trades.mean()
average_dynamic_weekly_trades = dynamic_strategy_weekly_trades.mean()

print(f"\nAverage Weekly Trades (Fixed-Threshold Strategy): {average_fixed_weekly_trades:.2f}")
print(f"Average Weekly Trades (Dynamic-Threshold Strategy): {average_dynamic_weekly_trades:.2f}")